In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2021-07-28 15:33:41.732337: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2021-07-28 15:33:43.259943: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-07-28 15:33:45.474614: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-28 15:33:45.475734: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-07-28 15:33:45.475822: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2021-07-28 15:33:45.537978: I tensorflow/stream_executor/platform/defa

In [2]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2021-07-28 15:33:45.693360: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2299995000 Hz


True

2021-07-28 15:33:45.694439: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5603394c3cf0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-07-28 15:33:45.694465: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-07-28 15:33:45.940685: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-28 15:33:45.941632: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56033b030760 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-07-28 15:33:45.941662: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2021-07-28 15:33:45.943270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


2021-07-28 15:33:49.496335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-28 15:33:49.497141: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-07-28 15:33:49.497230: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2021-07-28 15:33:49.497270: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.11
2021-07-28 15:33:49.497322: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2021-07-28 15:33:49.497373: I tensorflow/stream_executor/platform/defa

In [86]:
!pip install segmentation-models

In [87]:
import tensorflow as tf
from tensorflow import keras 

import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt


In [88]:
from keras.utils import conv_utils
from keras.backend import normalize_data_format
keras.backend.set_image_data_format('channels_last')
from keras.metrics import MeanIoU

In [8]:
ls

README.md                    img_process.ipynb  test_run2.ipynb
_image_classification.ipynb  test_run.ipynb     transfer_learning.ipynb


In [ ]:
# import zipfile
!unzip '/home/jupyter/prpls-dl/sub_reg_train128.zip' -d '/tmp'
!unzip '/home/jupyter/prpls-dl/sub_reg_label128.zip' -d '/tmp'

In [66]:
import fnmatch
import os

train_dirpath= '/tmp/sub_reg_train128'
print('Number of training images:', len(fnmatch.filter(os.listdir(train_dirpath), '*.tif')))

label_dirpath= '/tmp/sub_reg_label128'
print('Number of label images:', len(fnmatch.filter(os.listdir(label_dirpath), '*.tif')))

Number of training images: 8512
Number of label images: 8512


In [67]:
train_list=os.listdir('/tmp/sub_reg_train128')
mask_list=os.listdir('/tmp/sub_reg_label128')
mask = []
img = []
for filename in train_list:
    if filename.endswith('/tmp/sub_reg_train128/*.tif'):
        img.append(filename)

for filename in mask_list:
    if filename.endswith('/tmp/sub_reg_label128/*.tif'):
        mask.append(filename)

In [68]:
img.sort()
mask.sort()
img = img[:8512]
masks = mask[:8512]

In [69]:
y = np.zeros((8512, 128, 128), dtype=np.float32)
X = np.zeros((8512, 128, 128, 3), dtype=np.float32)

In [80]:
for file in img:
    index = img.index(i)
    dir_img = os.path.join(train_dirpath, i)
    img = Image.open(dir_img)
    X = img.append(img)
    
X = np.array(X)
    
for file in masks:
    dir_mask = os.path.join(label_dirpath, mask)
    mask_img = cv2.imread(dir_mask)
    mask_img = (mask!=2)*1.0
    mask_img = cv2.resize(mask, (128, 128))
    mask_img = 1.0*(mask[:,:,0]>0.2)
    y = mask.append(mask_img)
    
y = np.array(y)

In [93]:
print(X.shape)
print(y.shape)

(8512, 128, 128, 3)
(8512, 128, 128)


In [94]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [95]:
import tensorflow as tf
from tensorflow import keras 

import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import model_from_json

from tensorflow.keras.layers import Input, Conv2D, Reshape
from tensorflow.keras.models import Model

In [96]:
print("Class values in the dataset are ... ", np.unique(y_train))

Class values in the dataset are ...  [0.]


In [101]:
from tensorflow.keras.utils import to_categorical

n_classes=2
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))
print("Class values in the dataset are ... ", np.unique(y_train_cat))

Class values in the dataset are ...  [0. 1.]


In [102]:
import tensorflow as tf
from tensorflow import keras 

import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import model_from_json

from tensorflow.keras.layers import Input, Conv2D, Reshape
from tensorflow.keras.models import Model

from keras.layers import Reshape
N = X_train.shape[-1]

model = sm.Unet(backbone_name='resnet34', encoder_weights=None, input_shape=(None, None, N))

In [ ]:
# def dice_coefficient(y_true, y_pred):
#     numerator = 2 * tf.reduce_sum(y_true * y_pred)
#     denominator = tf.reduce_sum(y_true + y_pred)
#     return numerator / (denominator + tf.keras.backend.epsilon())

# def loss(y_true, y_pred):
#     return binary_crossentropy(y_true, y_pred) - tf.math.log(dice_coefficient(y_true, y_pred) + tf.keras.backend.epsilon())

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(optimizer='sgd', loss=sm.losses.JaccardLoss(), metrics=sm.metrics.IOUScore())
model.fit(X_train,y_train_cat,batch_size=32,epochs=100,validation_data=(X_test, y_test_cat))

Epoch 1/100
200/200 [==============================] - 65s 325ms/step - loss: 0.5221 - iou_score: 0.4779 - val_loss: 0.5217 - val_iou_score: 0.4783
Epoch 2/100
200/200 [==============================] - 64s 321ms/step - loss: 0.5213 - iou_score: 0.4787 - val_loss: 0.5209 - val_iou_score: 0.4791
Epoch 3/100
200/200 [==============================] - 64s 320ms/step - loss: 0.5205 - iou_score: 0.4795 - val_loss: 0.5201 - val_iou_score: 0.4799
Epoch 4/100
200/200 [==============================] - 64s 321ms/step - loss: 0.5197 - iou_score: 0.4803 - val_loss: 0.5193 - val_iou_score: 0.4807
Epoch 5/100
200/200 [==============================] - 64s 320ms/step - loss: 0.5190 - iou_score: 0.4810 - val_loss: 0.5187 - val_iou_score: 0.4813
Epoch 6/100
200/200 [==============================] - 64s 320ms/step - loss: 0.5184 - iou_score: 0.4816 - val_loss: 0.5181 - val_iou_score: 0.4819
Epoch 7/100
200/200 [==============================] - 64s 321ms/step - loss: 0.5178 - iou_score: 0.4822 - val_l

In [ ]:
model.save('resnet34_100epochs_128.tf')

In [ ]:
from keras.layers import Input, Conv2D
from keras.models import Model

from tensorflow.keras.utils import get_file
###Model 1
BACKBONE1 =  'vgg19'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

random_state=0
# define model
model1 = sm.Unet(BACKBONE1, encoder_weights=None, classes=n_classes, activation=activation)

# compile keras model with defined optimizer, loss and metrics
model1.compile(optimizer=optim, loss=sm.losses.JaccardLoss(), metrics=sm.metrics.IOUScore())
print(model1.summary())

earlystopping = callbacks.EarlyStopping(monitor ="val_loss", mode ="min", patience = 5, restore_best_weights = True)

import time
from datetime import datetime
start_time_resnet = datetime.now()
print('Start Time: {}'.format(start_time_resnet))


history1=model1.fit(X_train1, 
          y_train_cat,
          batch_size=64, 
          epochs=200,
          verbose=1,
          validation_data=(X_test1, y_test_cat), callbacks=[earlystopping])

end_time_resnet = datetime.now()
print('End Time: {}'.format(end_time_resnet))
print('Duration: {}'.format(end_time_resnet - start_time_resnet))

In [ ]:
model1.save('vgg19_200epochs128.tf')